# Median of Means Stopping Criteria

In [2]:
import qmcpy as qp
import numpy as np
import matplotlib.pyplot as plt


Integrals to test:
Keister
Box integral
Linear Function
Baysesian Logistic Regression
Genz Function 
Ishigami Function
Multimodal 2d
Asian Options
Other custom functions

In [ ]:
#mean of means vs. median of means

def test_momsc(dimension=2,num_samples_min=6,num_samples_max=18,m_max=10,number_trails=25,random_seed=11,integral_type="Keister",discrete_type="DigitalNetB2"):
    #parameters

    d = dimension
    N_min = num_samples_min 
    N_max = num_samples_max
    N_list = 2**np.arange(N_min,N_max)
    r = random_seed 
    num_trials = number_trails

    #errors

    error_median = np.zeros(N_max - N_min) 
    error_mean = np.zeros(N_max - N_min) 
    error_mean_onegen = np.zeros(N_max - N_min) 
    for i in range(num_trials):
        #lists
        y_median = []
        y_mean = []
        y_mean_one_gen = []
        #integral objects
        list_of_integrated_objects_random = []
        list_of_integrated_objects_default = []
        y_randomized_list = []
        y_default_list = []
        for k in range(r):
            #select discrete distribution type
            discrete = getattr(qp,discrete_type)
            distribution = discrete(d,random_seed,m_max)

            #select integrand, randomized distribution
            integral = getattr(qp,integral_type)
            integrand = integral(distribution)
            list_of_integrated_objects_random.append(integrand)
            x = integrand.discrete_distrib.gen_samples(N_list.max())
            y = integrand.f(x)
            y_randomized_list.append(y)

            #select integrand, vanilla distribution
            integrand = integral(discrete(d))
            list_of_integrated_objects_default.append(integrand)
            x = integrand.discrete_distrib.gen_samples(N_list.max())
            y = integrand.f(x)    
            y_default_list.append(y) 
                
        for N in N_list:

            y_median.append(np.median([np.mean(y[:N]) for y in y_randomized_list]))
            y_mean_one_gen.append(np.mean([np.mean(y[:N]) for y in y_default_list]))
            y_mean.append(np.mean([np.mean(y[:N]) for y in y_randomized_list]))

        answer = integrand.exact_integ(d)
        error_median += abs(answer-y_median)
        error_mean += abs(answer-y_mean)
        error_mean_onegen += abs(answer-y_mean_one_gen)

    #error calculation

    error_median /= num_trials
    error_mean /= num_trials
    error_mean_onegen /= num_trials

    #plotting

    plt.loglog(N_list,error_median,label = "median of means")
    plt.loglog(N_list,error_mean,label = "mean of means")
    plt.loglog(N_list,error_mean_onegen,label = "mean of random shifts")
    plt.xlabel("sample size")
    plt.ylabel("error")
    plt.title("Comparison of Digital Net Generators")
    plt.legend()
    plt.savefig("./meanvsmedian.png")
